In [24]:
import timm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
import torchvision
from torch.optim import Adam
import os
from PIL import Image

In [1]:
colab = False
if colab:
    !git clone https://github.com/KevivJaknap/Biomarkers-Prediction.git

In [2]:
train_healthy_dir = './database/train_data/healthy'
train_sick_dir = './database/train_data/sick'
test_healthy_dir = './database/test_data/healthy'
test_sick_dir = './database/test_data/sick'

if colab:
    train_healthy_dir = '/content/Biomarkers-Prediction/Trial 1/Thermogram Breast Cancer Detection/database/train_data/healthy'
    train_sick_dir = '/content/Biomarkers-Prediction/Trial 1/Thermogram Breast Cancer Detection/database/train_data/sick'
    test_healthy_dir = '/content/Biomarkers-Prediction/Trial 1/Thermogram Breast Cancer Detection/database/test_data/healthy'
    test_sick_dir = '/content/Biomarkers-Prediction/Trial 1/Thermogram Breast Cancer Detection/database/test_data/sick'


In [26]:
def get_images(path, label):
    ret = []
    for dir in os.listdir(path):
        dir += '/Segmentadas'
        for file in os.listdir(path+'/'+dir):
            ret.append((path+'/'+dir+'/'+file, label))
    
    return ret

In [27]:
train_data = get_images(train_healthy_dir, 0) + get_images(train_sick_dir, 1)
test_data = get_images(test_healthy_dir, 0) + get_images(test_sick_dir, 1)

In [28]:
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Usage:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomVerticalFlip, RandomRotation

transform = Compose([
    Resize((224, 224)),  # Resize images to 224x224
    RandomHorizontalFlip(p=0.5),  # Flip images horizontally,
    RandomRotation(degrees=10),  # Rotate images randomly,
    ToTensor(),  # Convert PIL image to tensor
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

def get_torch_dataset(dataset):
    return CustomImageDataset([i[0] for i in dataset], [i[1] for i in dataset], transform=transform)
# Assuming image_paths and labels are your data


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)
num_classes = 2  # Number of classes in your dataset
model.head = nn.Linear(model.head.in_features, num_classes)

model = model.to(device)
# Load the CIFAR10 dataset
train_dataset = get_torch_dataset(train_data)
test_dataset = get_torch_dataset(test_data)

# Define the data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

# Train the model
for epoch in range(10):  # Number of epochs
    running_loss = 0.0
    counter = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
        counter += 1
    print(f'Epoch: {epoch} | Loss: {running_loss/counter}')
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch: 1, loss: 0.754
Epoch: 1, loss: 14.090
Epoch: 1, loss: 14.270
Epoch: 1, loss: 2.106
Epoch: 1, loss: 3.155
Epoch: 1, loss: 0.670
Epoch: 1, loss: 0.757
Epoch: 1, loss: 0.671
Epoch: 1, loss: 0.489
Epoch: 1, loss: 0.586
Epoch: 1, loss: 0.545
Epoch: 1, loss: 0.501
Epoch: 1, loss: 0.425
Epoch: 1, loss: 0.743
Epoch: 1, loss: 0.433
Epoch: 1, loss: 0.457
Epoch: 1, loss: 0.623
Epoch: 1, loss: 0.493
Epoch: 1, loss: 0.657
Epoch: 1, loss: 0.328
Epoch: 1, loss: 0.258
Epoch: 1, loss: 0.548
Epoch: 1, loss: 0.371
Epoch: 1, loss: 0.263
Epoch: 1, loss: 0.498
Epoch: 1, loss: 0.591


KeyboardInterrupt: 